In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import Dataset, DataLoader
import pyarrow.parquet as pq
from torch.autograd import Variable
import pywt
import pywt.data
from scipy.signal import butter, lfilter 
from src.model import acl_net
from src.utils import *
#from tensorboardX import SummaryWriter

In [3]:
full_data = pq.read_pandas('../kaggle_data/train.parquet').to_pandas().values

In [ ]:
sig_pro = signal_processing.SignalProcess()
for j in range(5,6):
    data = full_data[:,j * 1000:(j + 1) * 1000]
    converted = np.apply_along_axis(sig_pro.butter_highpass_filter,0,data)
    converted = np.apply_along_axis(sig_pro.denoise_signal, 0, converted[200000:])
    #converted  = np.apply_along_axis(hard_thres,0,converted)
    converted = np.apply_along_axis(corona_denoise,0,converted)
    break
    if j == 0:
        convert = converted
    else:
        convert = np.hstack([convert,converted])

In [ ]:
meta_data = pd.read_csv("../ken_hayashima1989/metadata_train.csv")[5000:6000]

In [ ]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = acl_net.ACLNet().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = 5e-4)

In [ ]:
converted = np.vstack([converted[:204800,:],converted[-204800:,:]])
agd = data_augmentor.AugmentedData(converted,meta_data)
agd = DataLoader(agd,batch_size = 64,shuffle = True)

In [ ]:
for epoch in range(100):
    for i ,sample_batched in enumerate(agd_data):
        model.zero_grad()
        #model.hidden = model.init_hidden()
        signals = Variable(sample_batched['X'].float().to(device)) / 50
        signals = signals.permute(0,2,1)
        y_a = sample_batched['Y_A'].float().to(device)
        y_b = sample_batched['Y_B'].float().to(device)
        lam = sample_batched['lam'].float().to(device)
        #print(signals.size())
        #print(signals.size())

        output = torch.squeeze(model(signals))
        #print(output.size())
        
        #print(lam)
        #y_a_loss = criterion(output, y_a)
        #y_b_loss = criterion(output, y_b)
        #print(y_a_loss)
        loss = mixup_loss(output,y_a,y_b,lam)
        #print(loss)
        #writer.add_scalar('loss',loss)
        if output.size()[0] != 64 and epoch % 5 == 0:
            print(loss)
        loss.backward()
        optimizer.step()
        

In [ ]:
def mixup_loss(output,y_a,y_b,lam):
    y_a_loss = y_a * torch.log(output + 1e-8) + (1 - y_a) * torch.log(1 - output + 1e-8)
    y_b_loss = y_b * torch.log(output + 1e-8) + (1 - y_b) * torch.log(1 - output + 1e-8)
    return - torch.mean(lam * y_a_loss + (1 - lam) * y_b_loss)

In [ ]:
sig_pro = SignalProcess()
for j in range(64):
    validation = full_data[:,j * 100:(j + 1) * 100]
    validation_converted = np.apply_along_axis(sig_pro.butter_highpass_filter,0,validation)
    validation_converted = np.apply_along_axis(sig_pro.denoise_signal, 0, validation_converted[200000:])
    validation_converted = np.apply_along_axis(corona_denoise,0,validation_converted)
    converted = np.vstack([validation_converted[:204800,:],validation_converted[-204800:,:]])
    converted = converted.T
    converted = converted[:,None,:] 
    converted = torch.from_numpy(converted).float().to(device) / 50
    output = torch.squeeze(model(converted)).cpu().data.numpy()
    output = np.round(output).astype(int)
    if j == 0:
        outputs = output
    else:
        outputs = np.concatenate([outputs,output])
ids = np.array(range(8712 + 14000,29049))
test_output = pd.DataFrame({'signal_id':ids,'target':outputs})
test_output.to_csv('final_ids.csv',index=None)
    
'''
test = pq.read_pandas('../ken_hayashima1989/train.parquet').to_pandas().values
for j in range(6):
    #data = test[:,:(j + 1) * 500]
    converted = np.apply_along_axis(sig_pro.butter_highpass_filter,0,test)
    converted = np.apply_along_axis(sig_pro.denoise_signal, 0, converted[200000:])
    #converted  = np.apply_along_axis(hard_thres,0,converted)
    converted = np.apply_along_axis(corona_denoise,0,converted)
'''

In [ ]:
model.eval()

In [ ]:
pd.read_csv('../ken_hayashima1989/sample_submission.csv')

In [ ]:
final_id = pd.concat([pd.read_csv('first_7000_ids.csv'),pd.read_csv('second_7000_ids.csv'),pd.read_csv('final_ids.csv')])

In [ ]:
final_id.to_csv('ken_first_submission.csv',index=None)